## Курсовой проект по курсу "Машинное обучение в бизнесе"

### Проверка работоспособности API

In [1]:
import pandas as pd
import urllib.request
import json 
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

def get_prediction(field_array=None, myurl=None):
    """
    Input parameters:
        field_array: python list - contains raw fields needed to make a prediction
        myurl: REST API url
    Returns:
        result: float - model prediction
    """
    result = 0
    age, sex, cp, exng, trtbps, thalachh = field_array
    
    body = {"age": age,
            "sex": sex,
            "cp": cp,
            "exng": exng,
            "trtbps": trtbps,
            "thalachh": thalachh}

    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = urllib.request.urlopen(req, jsondataasbytes)
    
    return json.loads(response.read())['predictions']

API_URL = 'http://0.0.0.0:8180/predict'

X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

/Users/annasmelova/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


Посмотрим что за данные у нас есть

In [2]:
X_test[['age', 'sex', 'cp', 'exng', 'trtbps', 'thalachh']].head(3)

,age,sex,cp,exng,trtbps,thalachh
0,70,1,0,1,145,125
1,64,1,3,0,170,155
2,59,1,3,0,170,159


In [3]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
age         61 non-null int64
sex         61 non-null int64
cp          61 non-null int64
exng        61 non-null int64
trtbps      61 non-null int64
thalachh    61 non-null int64
dtypes: int64(6)
memory usage: 3.0 KB


### Getting predictions from REST API

In [4]:
%%time
predictions = X_test[['age', 'sex', 'cp', 'exng', 'trtbps', 
                      'thalachh']].apply(lambda x: get_prediction(x, API_URL), 1)

CPU times: user 63 ms, sys: 18.6 ms, total: 81.5 ms
Wall time: 1.7 s


check performance (roc auc)

In [5]:
roc_auc_score(y_score=predictions.values, y_true=y_test)

0.8725490196078431